In [1]:
from SimPy.Simulation import *
import random
import numpy as np
import math
import read_data as rd
import scipy
import scipy.stats as st

In [2]:
import os
from pathlib import Path

filenames = [str(Path(os.path.abspath('')).parent) + "\Data Collection\KevinYeData.txt",
            str(Path(os.path.abspath('')).parent) + "\Data Collection\dataviviandong.txt",
            str(Path(os.path.abspath('')).parent) + "\Data Collection\PatrickQData.txt",
            str(Path(os.path.abspath('')).parent) + "\Data Collection\TamaHoareData.txt"]
results = rd.ReadData(filenames=filenames)
AllArrivals = results[0]
AllServices = results[1]

In [13]:
def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return lower, upper

In [14]:
#From our analysis, it suggests that Pearson3 is the best in approximating ‘inter-arrival time’ Data.
# Pearson3 Distribution Parameters
P3skew,P3loc,P3scale = st.pearson3.fit(AllArrivals)
print(P3skew,P3loc,P3scale)

2.43659632424228 89.15123787553043 108.61278925458328


In [21]:
# For our 'service times' data, it suggests that beta distribution is the best fit for the data.
# Beta Distribution Parameters
beta_a, beta_b,beta_loc,beta_scale = st.beta.fit(AllServices)
print(beta_a,beta_b,beta_loc,beta_scale)

1.4923327932636143 15.801670930407775 7.642594962182772 1311.3920705953165


In [15]:
class Source(Process):
    """generate random arrivals"""
    def run(self, N):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run())
            t = abs(st.pearson3.rvs(skew=P3skew,loc=P3loc,scale=P3scale))
            yield hold, self, t

In [16]:
class Arrival(Process):
    """an arrival"""
    n=0 

    def run(self):
        Arrival.n +=1
        
        arrivetime = now()
        
        G.numbermon.observe(Arrival.n)
        
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
            
        yield request, self, G.server
        yield hold, self, st.beta.rvs(a=beta_a,b=beta_b,loc=beta_loc,scale=beta_scale)
        yield release, self, G.server
        
        Arrival.n -=1
        
        G.numbermon.observe(Arrival.n)
        
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
            
        delay = now()-arrivetime
        G.delaymon.observe(delay)

In [17]:
class G:
    server = 'dummy'
    delaymon = 'Monitor' # W
    numbermon = "Monitor" # L
    busymon = 'Monitor' # B

In [18]:
def model(N, maxtime, rvseed,K):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(K)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
    G.busymon = Monitor()

    Arrival.n = 0
    
    # simulate
    s = Source('Source')
    activate(s, s.run(N))
    simulate(until=maxtime)

    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    B = G.busymon.timeAverage()
    
    return W, L, B

The LAB Cafe is open from 7.30am to 3pm a total of 7.5 hours per day which is equivalent to 27000 seconds which would be the maxtime of each simulation.

In [23]:
## Experiment ----------------


for Kap in [3]:
    allW = []
    allL = []
    allB = []
    allLambdaEffective = []

    for k in range(50):
        seed = 123*k
        result = model(N=10000, maxtime=27000, rvseed=seed, K=Kap)
        allW.append(result[0])
        allL.append(result[1])
        allB.append(result[2])
        allLambdaEffective.append(result[1]/result[0])


    print("=================")
    print("When K=",Kap)
    print("    Estimate of W:", np.mean(allW))
    print("    Conf int of W:", conf(allW))
    print("    Estimate of L:", np.mean(allL))
    print("    Conf int of L:", conf(allL))
    print("    Estimate of B:", np.mean(allB))
    print("    Conf int of B:", conf(allB))
    print("    Estimate of LambdaEffective:", np.mean(allLambdaEffective))
    print("    Conf int of LambdaEffective:", conf(allLambdaEffective))

When K= 3
    Estimate of W: 143.28751362199432
    Conf int of W: (140.62749857595333, 145.94752866803532)
    Estimate of L: 1.6283817237073634
    Conf int of L: (1.5753783867711864, 1.6813850606435405)
    Estimate of B: 0.7140026917524969
    Conf int of B: (0.7037947894617138, 0.7242105940432799)
    Estimate of LambdaEffective: 0.011347266899941985
    Conf int of LambdaEffective: (0.011098053844956974, 0.011596479954926997)
